In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

df_1 = pd.read_csv("../output/task2-1.csv")
df_2 = pd.read_csv("../output/task2-2.csv")
df_2_total = pd.read_csv("../output/task2-2-total.csv")

In [2]:
# plot the number of cells over time
plt.figure(figsize=(8, 3))
plt.plot(df_1['time'], df_1['cells'], label='Cells')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-1.pdf', bbox_inches='tight')
plt.close()

In [3]:
plt.figure(figsize=(3, 3))
plt.plot(df_2['x'], df_2['y'], label='_task2-2', color='gray')
plt.plot(df_2['x'][0], df_2['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_2['x'].iloc[-1], df_2['y'].iloc[-1], 'rx', label='End', ms=10, mew=2)
plt.xlim(25,75)
plt.ylim(25,75)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig('../../report/graphics/task2-2.pdf', bbox_inches='tight')
plt.close()

In [4]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-m"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-m/{file}")
        # log to base 10 for power
        record["m"] = int(name[8:])
        data[name] = record

for name, record in data.items():
    dt = 0.001
    record['df']['time'] = record['df']['time'] * dt

In [5]:
# plot the number of cells over time for all m
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=('$10^{' + str(record["m"]) + '}$'))
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Capacity')
plt.yscale('log')
plt.ylim(1e9, 1e17)
plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')

/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_70532/2453372918.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')
/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_70532/2453372918.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')


In [14]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-h"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-h/{file}")
        # log to base 10 for power
        record["h"] = int(name[10:])
        data[name] = record

# scale time
for name, record in data.items():
    dt = 10 ** record['h']
    record['df']['time'] = record['df']['time'] * dt

# sort by h
data = dict(sorted(data.items(), key=lambda x: x[1]['h'], reverse=True))

# solve analytically first
x = np.linspace(0, 1200, 2000)
y = 10 ** (13 - (4 * (np.e ** (-0.006 * x))))
analytical = pd.DataFrame({'time': x, 'cells': y})

# calculate the error to the analytical solution for all h using root mean square percentage error
for name, record in data.items():
    record['error'] = np.sqrt(np.mean((record['df']['cells'] - np.interp(record['df']['time'], analytical['time'], analytical['cells'])) ** 2)) / np.mean(record['df']['cells']) * 100

# print in table format
print("h & Error")
for name, record in data.items():
    # round to 5 significant figures
    print(f"{10 ** record['h']} & {record['error']:.4}\% \\\\")
print("Analytical & 0.0 \\\\")
print()
    

h & Error
100 & 66.69\% \\
10 & 5.296\% \\
1 & 0.5238\% \\
0.1 & 0.05233\% \\
0.01 & 0.005233\% \\
0.001 & 0.0005239\% \\
0.0001 & 5.82e-05\% \\
Analytical & 0.0 \\



In [7]:

# plot the number of cells over time for all h
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=f'{10 ** record["h"]}')
plt.plot(analytical['time'], analytical['cells'], label='Analytical')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Dt')
plt.savefig('../../report/graphics/task2-1-h.pdf', bbox_inches='tight')

In [8]:
# plot again but zoomed in ar 450-550
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=f'{10 ** record["h"]}')
plt.plot(analytical['time'], analytical['cells'], label='Analytical')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Dt')
plt.xlim(450, 550)
plt.ylim(0.55e13, 0.65e13)
plt.savefig('../../report/graphics/task2-1-h-zoom.pdf', bbox_inches='tight')
plt.close()

In [9]:
# plot the number of cells over time
plt.figure(figsize=(8, 3))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total.pdf', bbox_inches='tight')

In [10]:
# plot the number of cells over time
plt.figure(figsize=(8, 3))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')


# estimate a linear fit
x = df_2_total['time']
y = df_2_total['cells']

A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]
print(f"m={m}, c={c}")

# dashed
plt.plot(x, m*x + c, 'r', label='Fitted line', linestyle='dashed')

# add text with the equation
plt.text(0, 0.9e15, f'y = {m:.4g}x + {c:.2g}', fontsize=12)

plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total-linear.pdf', bbox_inches='tight')

m=8290757.139360582, c=0.10363446464054554


In [13]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-t"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-t/{file}")
        record["t"] = int(name[8:])
        print(record["t"])
        data[name] = record

# scale time
for name, record in data.items():
    dt = 0.001
    record['df']['time'] = record['df']['time'] * dt

# sort by t (in reverse order)
data = dict(sorted(data.items(), key=lambda x: x[1]['t'], reverse=True))

# get the final number of cells for each t
final_cells = []
for name, record in data.items():
    final_cells.append(record['df']['cells'].iloc[-1])

# calculate the percentage full using 10^13 as the full capacity
full_capacity = 10 ** 13
percentage_full = [x / full_capacity for x in final_cells]

# print in table format
print("T & Cells & \% Full \\\\")
for i, (name, record) in enumerate(data.items()):
    print(f"{record['t']} & {final_cells[i]:.2e} & {percentage_full[i]:.4f}\% \\\\")
print()


2000
1800
1200
1000
1400
1600
T & Cells & \% Full \\
2000 & 1.00e+13 & 0.9999\% \\
1800 & 1.00e+13 & 0.9998\% \\
1600 & 9.99e+12 & 0.9994\% \\
1400 & 9.98e+12 & 0.9979\% \\
1200 & 9.93e+12 & 0.9931\% \\
1000 & 9.77e+12 & 0.9774\% \\



In [12]:
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=record["t"])
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.yscale('log')
plt.savefig('../../report/graphics/task2-1-t.pdf', bbox_inches='tight')